***Smartcab Project Report ***
==============================

by Moritz Bendiek






Implement a basic driving agent
-------------------------------

The requested behavior (random choice of action disregarding any input information) is realized by adding the following code to the `update` method:
```python
action_list = [None, 'forward', 'left', 'right']
action = action_list[random.randint(0,3)]
```

Note that `random` should be initialized (via `seed`) first.

**QUESTION:** Observe what you see with the agent's behavior as it takes random actions. Does the **smartcab** eventually make it to the destination? Are there any other interesting observations to note?

**ANSWER**: The agent executes a "random walk". Due to the limited size of the world model (6*8=48 positions on the grid), the smartcab will arrive at its destination within a reasonably short time span. I executed 100 test runs and found that the smartcab arrived at the destination before the hard time limit (-100) was reached 63 times, and 37 times the agent ran into the hard time limit.

The smartcab apparently is influenced by the traffic lights, insofar as it does not always move when it sees a "red" light. However, judging from the visualization, I am not sure that this behavior is consistent. Also, the movement of the smartcab somedoes does not seem to fit to the chosen action (e.g. displays 'left' as action, but drives forward). 

Inform the driving agent
------------------------

**TASK:** Now that your driving agent is capable of moving around in the environment, your next task is to identify a set of states that are appropriate for modeling the smartcab and environment. The main source of state variables are the current inputs at the intersection, but not all may require representation. You may choose to explicitly define states, or use some combination of inputs as an implicit state. At each time step, process the inputs and update the agent's current state using the `self.state` variable. Continue with the simulation deadline enforcement `enforce_deadline` being set to `False`, and observe how your driving agent now reports the change in state as the simulation progresses.

**QUESTION:** What states have you identified that are appropriate for modeling the smartcab and environment? Why do you believe each of these states to be appropriate for this problem?

**ANSWER:** While intuitively the position of the smartcab on the grid should be part of its state, I think I can omit it as in order to learn to drive it is not relevant for the smartcab where it is, as all traffic rules apply equally in all places.

To differentiate between situations that require different behavior of the agent, it is sufficient to regard a few sub-states defined based on the inputs / destination:
* In which direction shall the smartcab move? Here, we can use `self.next_waypoint` directly. It can be any of `['left', 'forward', 'right']`
* Is the smartcab allowed to move left? This is either `True` or `False` and can be determined according to the know traffic rules: `True` if `light` is `green` and `oncoming` is either `forward` of `left`, else `False`.
* Is the smartcab allowed to move forward? This is `True` if `light` is `green`, else it is `False`.
* Is the smartcab allowed to move right? This is `False` if `left` is `forward` and else `True`.

Therefore I choose to model the state as follows:
``` python
left_allowed = (inputs['light']=='green') & (inputs['oncoming'] in ['left', None])
forward_allowed = inputs['light']=='green'
right_allowed = (inputs['light']=='green') | (inputs['left'] in [None, 'left', 'right'])
self.state = [self.next_waypoint, left_allowed, forward_allowed, right_allowed]
```




**OPTIONAL:** How many states in total exist for the smartcab in this environment? Does this number seem reasonable given that the goal of Q-Learning is to learn and make informed decisions about each state? Why or why not?

**ANSWER:** The total number of states is the Cartesian Product of all the dimensions, in this case 3 \* 2 \* 2 \* 2 = 24 states.

The goal of Q-Learning is to move through all states *very* often. As in the model there are 100 test runs, and each tests run consists of a capped number of individual moves (even with `enforce_deadline` being set to `False`, a run stops after hitting the hard deadline of -100), we can broadly assume that with 24 moves per run, we will visit 2400 states. However it is likely that the probability to be in a specific state can vary (i.e. is not uniform).  


Implement a Q-Learning Driving Agent
------------------------------------

**TASK:** With your driving agent being capable of interpreting the input information and having a mapping of environmental states, your next task is to implement the Q-Learning algorithm for your driving agent to choose the best action at each time step, based on the Q-values for the current state and action. Each action taken by the smartcab will produce a reward which depends on the state of the environment. The Q-Learning driving agent will need to consider these rewards when updating the Q-values. Once implemented, set the simulation deadline enforcement `enforce_deadline` to `True`. Run the simulation and observe how the smartcab moves about the environment in each trial.

The formulas for updating Q-values can be found in this video.

**QUESTION:** What changes do you notice in the agent's behavior when compared to the basic driving agent when random actions were always taken? Why is this behavior occurring?

Improve the Q-Learning Driving Agent
------------------------------------

Your final task for this project is to enhance your driving agent so that, after sufficient training, the smartcab is able to reach the destination within the allotted time safely and efficiently. Parameters in the Q-Learning algorithm, such as the learning rate (`alpha`), the discount factor (`gamma`) and the exploration rate (`epsilon`) all contribute to the driving agent’s ability to learn the best action for each state. To improve on the success of your smartcab:

* Set the number of trials, `n_trials`, in the simulation to 100.
* Run the simulation with the deadline enforcement `enforce_deadline` set to `True` (you will need to reduce the update delay `update_delay` and set the `display` to `False`).
* Observe the driving agent’s learning and smartcab’s success rate, particularly during the later trials.
* Adjust one or several of the above parameters and iterate this process.

This task is complete once you have arrived at what you determine is the best combination of parameters required for your driving agent to learn successfully.

**QUESTION:** Report the different values for the parameters tuned in your basic implementation of Q-Learning. For which set of parameters does the agent perform best? How well does the final driving agent perform?

**QUESTION:** Does your agent get close to finding an optimal policy, i.e. reach the destination in the minimum possible time, and not incur any penalties? How would you describe an optimal policy for this problem?